<a href="https://colab.research.google.com/github/nicolas998/Hydrology_course/blob/master/04_Rating_curve_construction_from_USGS_field_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import required packages

In [1]:
#In order, import packages to manage data as tables (pd), plot (plt), and operate with matrices (np)
import pandas as pd
import pylab as plt
import numpy as np
import requests
#Package to make interactive plots
import plotly.express as px
#Package to perfrom regressions
import scipy.optimize as opt

# Read USGS field measurements of stage and discharge

The data is downloaded from the NWIS web page corresponding to a given gauge.

To access to it go to the following link:

https://waterwatch.usgs.gov/?id=ww_current

From there, search your region of interest and finally the guage that you will be using for the workshop. Once you click on it, the USGS web page of the gauge shoul look like the one on the following link:

https://waterdata.usgs.gov/monitoring-location/USGS-02030500/

In it, you may change the code at the end for the code corresponding to the USGS  gauge that you want to work with. Once in the page do the following steps:
1. Scroll down until you see the field measurements brown box.
2. Click on **Show these data types**
3. Click on the **Graph it** button corresponding to *Gage height, feet*
4. Scroll up until you see a figure with a few dots.
5. click on the button on top of the figure named **period of record**. The figure should now have way more dots.
6. Click on the **Download data** button that is below the figure and then on the radial button **field measurements** that is below the last one and finally click on the blue button called **Retrieve**.
7. Congrats!! you downloaded the field measurementes corresponding to the gauge. Now o back to step **3** and select the *Discharge, cubic feet per second* variable instead and repeat the steps.

By the end of the steps, you should have two csv files one with the satage values (name it stage.csv) and other with the discharge (discharge.csv)

After that, upload them into Google colab or your google drive.
- **Google Colab**: click on the folder icon on the left bar and move the csv file from your local folder to the space that showed up when you clicked on the folder.
- **Drive**: Is basically the same with the difference that first you have to connect your drive to your colab environment. To do that, click on the folder icon on the left, then click on the folder icon that has a drive icon on its corner and then click on accpet. taht will connect your drive. Then, just explore it and upload the data to your desired folder.


In [2]:
#Define function that will read the csv files downloaded from the NWIS web page
#Caution! a function on its own does nothing, here we are only deifning it, we will called it later in that moment it will actually work!
def read_usgs_rating_curve_data_csv(path):
  #Read the file
  df = pd.read_csv(path)
  #Get only the values and the dates
  df = df[['time','value']]
  #Set the dates as the index
  df.loc['time'] = pd.to_datetime(df['time'])
  df = df.set_index('time')
  df.sort_index(inplace=True)
  #Drop empty dates
  df = df[df.index.notna()]
  #Return the variable
  return df

#Function to download USGS historical records
def fetch_usgs_streamflow_data(site_code, start_date, end_date):
    """
    Fetches USGS streamflow data for a given site and date range.

    Parameters:
    - site_code (str): USGS site code (e.g., '03339000')
    - start_date (str): Start date in format 'YYYY-MM-DD'
    - end_date (str): End date in format 'YYYY-MM-DD'

    Returns:
    - pd.DataFrame: Dataframe with datetime index and a column for streamflow in cubic meters per second (cms)
    """
    url = (
        f"https://waterservices.usgs.gov/nwis/iv/?format=json&sites={site_code}"
        f"&startDT={start_date}T00:00:00&endDT={end_date}T23:59:59&parameterCd=00060"
        f"&siteStatus=all"
    )

    response = requests.get(url)
    data = response.json()

    # Extract time-series data
    timeseries = data.get("value", {}).get("timeSeries", [])

    records = []
    for series in timeseries:
        for entry in series["values"][0]["value"]:
            timestamp = entry["dateTime"]
            value = float(entry["value"]) if entry["value"] != "" else None
            if value is not None:
                value *= 0.0283168  # Convert cfs to cms
            records.append([timestamp, value])

    # Convert to DataFrame
    df = pd.DataFrame(records, columns=["Datetime", "Flow(cms)"])
    df["Datetime"] = pd.to_datetime(df["Datetime"], utc = True)
    df.set_index("Datetime", inplace=True)

    return df


## Using the function to read files and merge them

## Paths to the cvs files

Now that you can see your cvs files from colab, you should be able to read them. To do that, just find them on the left file explorer and right click on them, from there click on the **Copy path** option. Then, just replace the lon path that is on the following cell by the path that you copy. First do it for the stage and then for the discharge.

In [22]:
#read the stage csv
stage = read_usgs_rating_curve_data_csv('/content/drive/MyDrive/2025 Hydrology and Hydraulics/Example_data/CedarRiver-field-measurements_gauge.csv')
#read the discharge csv
discharge = read_usgs_rating_curve_data_csv('/content/drive/MyDrive/2025 Hydrology and Hydraulics/Example_data/CedarRiver-field-measurements_discharge.csv')
#merge them
idx = discharge.index.intersection(stage.index)
rating_curve_raw = pd.DataFrame({'stage': stage.loc[idx, 'value'], 'discharge': discharge.loc[idx, 'value']})

## A simple plot of the data

The following is an interactive plot of the data (don't get scared by the code it is all AI-generated! - Just click on the play icon on the left upper corner). Then, play with the buttons that allow you to change between linear and log scales



In [23]:
fig = px.scatter(rating_curve_raw, x='stage', y='discharge',
                 title='Interactive Stage-Discharge Relationship',
                 labels={'stage': 'Stage [feet]', 'discharge': 'Discharge [cfs]'},
                 hover_name=rating_curve_raw.index,
                 height=600)

fig.update_traces(marker=dict(size=8, line=dict(width=1, color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.update_layout(
    xaxis_title_font_size=16, yaxis_title_font_size=16,
    xaxis_tickfont_size=12, yaxis_tickfont_size=12,
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=0.01,
            y=1.1,
            showactive=True,
            buttons=list([
                dict(label="Linear X",
                     method="relayout",
                     args=[{"xaxis.type": "linear"}]),
                dict(label="Log X",
                     method="relayout",
                     args=[{"xaxis.type": "log"}])
            ])
        ),
        dict(
            type="buttons",
            direction="right",
            x=0.25,
            y=1.1,
            showactive=True,
            buttons=list([
                dict(label="Linear Y",
                     method="relayout",
                     args=[{"yaxis.type": "linear"}]),
                dict(label="Log Y",
                     method="relayout",
                     args=[{"yaxis.type": "log"}])
            ])
        )
    ]
)
fig.show()

## To do: Baseed on the previous plot that compares stage and discharge, answer the following questions:
- Is the relationship monotonic?
- Do you see outliers?
- Do you see evidence of multiple rating segments?
- Is it useful to change the axis scales?


## To do: Make a plot comparing the stage (y-axis) vs the time (x-axis) using the data in the variable *rating_curve_raw*

## To do: Make a plot comparing the discharge (y-axis) vs the time (x-axis) using the data in the variable *rating_curve_raw*

# The rating curve construction

The following is a function made to identify the rating curve equation:

$Q = a \cdot (h-h_0)^b$

where $a$, $h_0$, and $b$ have to be found.

In [24]:
import numpy as np
from dataclasses import dataclass

@dataclass
class RatingFit:
    a: float
    b: float
    h0: float
    rmse_log10: float
    r2_log10: float
    n: int

def fit_rating_curve_powerlaw(
    h: np.ndarray,
    Q: np.ndarray,
    h0_bounds: tuple[float, float] | None = None,
    h0_steps: int = 400,
    min_points: int = 10,
    eps: float = 1e-12,
) -> RatingFit:
    """
    Fit Q = a*(h - h0)^b using a grid search over h0 and log-log linear regression for a,b.

    Parameters
    ----------
    h, Q : arrays
        Stage (gage height) and discharge; must be same length.
    h0_bounds : (low, high) or None
        Search range for h0. If None, uses (min(h)-range_pad, min(h)-small_pad).
    h0_steps : int
        Number of candidate h0 values in the grid search.
    min_points : int
        Minimum number of valid points required (h > h0 and Q > 0).
    eps : float
        Small value to avoid log(0).

    Returns
    -------
    RatingFit
        Best-fit parameters and log-space diagnostics.
    """
    h = np.asarray(h, dtype=float).ravel()
    Q = np.asarray(Q, dtype=float).ravel()

    if h.size != Q.size:
        raise ValueError("h and Q must have the same length.")
    if h.size < min_points:
        raise ValueError(f"Need at least {min_points} points; got {h.size}.")

    # Filter gross invalids
    m0 = np.isfinite(h) & np.isfinite(Q) & (Q > 0)
    h, Q = h[m0], Q[m0]
    if h.size < min_points:
        raise ValueError(f"After filtering Q>0 and finite values, only {h.size} points remain.")

    hmin = float(np.min(h))
    hrng = float(np.max(h) - hmin)

    if h0_bounds is None:
        # USGS-ish heuristic: h0 is usually slightly below min observed stage.
        low = hmin - 0.5 * max(hrng, 1e-6)
        high = hmin - 1e-6
        h0_bounds = (low, high)

    h0_grid = np.linspace(h0_bounds[0], h0_bounds[1], h0_steps)

    best = None
    best_rmse = np.inf

    log10 = np.log10

    for h0 in h0_grid:
        x = h - h0
        m = x > 0
        if np.count_nonzero(m) < min_points:
            continue

        # log10(Q) = log10(a) + b*log10(h - h0)
        X = log10(x[m] + eps)
        Y = log10(Q[m] + eps)

        # Simple OLS
        Xc = X - X.mean()
        denom = np.dot(Xc, Xc)
        if denom <= 0:
            continue

        b = np.dot(Xc, (Y - Y.mean())) / denom
        loga = Y.mean() - b * X.mean()
        a = 10 ** loga

        Yhat = loga + b * X
        resid = Y - Yhat
        rmse = float(np.sqrt(np.mean(resid**2)))

        if rmse < best_rmse:
            # R^2 in log space
            ss_res = float(np.sum(resid**2))
            ss_tot = float(np.sum((Y - Y.mean())**2))
            r2 = 1.0 - ss_res / ss_tot if ss_tot > 0 else np.nan

            best_rmse = rmse
            best = RatingFit(
                a=float(a),
                b=float(b),
                h0=float(h0),
                rmse_log10=rmse,
                r2_log10=float(r2),
                n=int(np.count_nonzero(m)),
            )

    if best is None:
        raise RuntimeError("Could not fit: no h0 candidate yielded enough valid points.")

    return best


Following, we implement the function to fit the rating curve

In [25]:
curve_parameters = fit_rating_curve_powerlaw(rating_curve_raw['stage'].values, rating_curve_raw['discharge'].values, h0_steps=800)

In [26]:
curve_parameters

RatingFit(a=495.7916778912616, b=1.8459026072858224, h0=1.5309501977471829, rmse_log10=0.10190717967272561, r2_log10=0.966517901364861, n=1051)

Now, we define a function to estimate $Q$ using the fitted parameters: $a$, $h_0$, and $b$

In [27]:
def predict_Q(h, a, b, h0):
    x = np.maximum(h - h0, 0.0)
    return a * x**b

Then, we just evaluate the function for our the range of values:

In [28]:
# Range of values between the minimum and the maximum
stage_range = np.linspace(rating_curve_raw['stage'].min(), rating_curve_raw['stage'].max(), 100)
#Fitted Q
qfit_range = predict_Q(stage_range, curve_parameters.a, curve_parameters.b, curve_parameters.h0)

## To do: Make a plot comparing the rating cuvre that you got using the fittiing and the raw data.
- Do you notice any big differences?
- Are differences higher for the low or the high values? Explain
- Would you trust this equation to estimate discharge? Explain.

In [29]:
import plotly.graph_objects as go

fig = go.Figure()

# Add original data points
fig.add_trace(go.Scatter(
    x=rating_curve_raw['stage'],
    y=rating_curve_raw['discharge'],
    mode='markers',
    name='Field Measurements',
    marker=dict(size=8, color='blue', line=dict(width=1, color='DarkSlateGrey'))
))

# Add fitted curve
fig.add_trace(go.Scatter(
    x=stage_range,
    y=qfit_range,
    mode='lines',
    name='Fitted Curve',
    line=dict(color='red')
))

fig.update_layout(
    title='Interactive Stage-Discharge Relationship with Fitted Curve',
    xaxis_title='Stage [feet]',
    yaxis_title='Discharge [cfs]',
    xaxis_title_font_size=16, yaxis_title_font_size=16,
    xaxis_tickfont_size=12, yaxis_tickfont_size=12,
    height=600,
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=0.01,
            y=1.1,
            showactive=True,
            buttons=list([
                dict(label="Linear X",
                     method="relayout",
                     args=[{"xaxis.type": "linear"}]),
                dict(label="Log X",
                     method="relayout",
                     args=[{"xaxis.type": "log"}])
            ])
        ),
        dict(
            type="buttons",
            direction="right",
            x=0.25,
            y=1.1,
            showactive=True,
            buttons=list([
                dict(label="Linear Y",
                     method="relayout",
                     args=[{"yaxis.type": "linear"}]),
                dict(label="Log Y",
                     method="relayout",
                     args=[{"yaxis.type": "log"}])
            ])
        )
    ]
)

fig.show()

## To do: Evaluate how good your fit is, plot the residual error and based on it answer the following questions:
- When does the error becomes larger?
- Why do you consider that errors behave like that?
- Suggest a practical solution to improve the error adjustment.

In [30]:
import plotly.express as px

# Evaluate the fitted curve at the raw stage points
q_predicted_at_raw_stages = predict_Q(
    rating_curve_raw['stage'].values,
    curve_parameters.a,
    curve_parameters.b,
    curve_parameters.h0
)

# Calculate the difference (residuals)
residuals = 100*(rating_curve_raw['discharge'] - q_predicted_at_raw_stages)/rating_curve_raw['discharge'].mean()

# Create a DataFrame for plotting residuals
residuals_df = pd.DataFrame({
    'stage': rating_curve_raw['stage'],
    'residuals': residuals,
    'time': rating_curve_raw.index
})

# Plot the residuals against stage
fig_residuals = px.scatter(
    residuals_df,
    x='stage',
    y='residuals',
    title='Residuals vs. Stage',
    labels={'stage': 'Stage [feet]', 'residuals': 'Relative Residuals [%]'},
    hover_name='time',
    height=600
)

fig_residuals.update_traces(marker=dict(size=8, opacity=0.7))
fig_residuals.update_layout(
    xaxis_title_font_size=16, yaxis_title_font_size=16,
    xaxis_tickfont_size=12, yaxis_tickfont_size=12
)

fig_residuals.show()

# Rating Curve compared with field data

In [13]:
# Example usage
site_code = "02030500"  # Replace with your desired site code
start_date = "2000-01-01"  # Replace with your desired start date
end_date = "2026-01-31"  # Replace with your desired end date

streamflow_data = fetch_usgs_streamflow_data(site_code, start_date, end_date)

In [14]:
streamflow_data.head()

,Flow(cms)
Datetime,
2005-06-23 04:00:00+00:00,1.783958
2005-06-23 04:15:00+00:00,1.783958
2005-06-23 04:30:00+00:00,1.783958
2005-06-23 04:45:00+00:00,1.783958
2005-06-23 05:00:00+00:00,1.783958


In [ ]:
streamflow_data['Flow(cfs)'] = streamflow_data['Flow(cms)'] / 0.0283168

In [20]:
rating_curve_raw['discharge'].max()

143000

In [21]:
streamflow_data.max()

,0
Flow(cfs),12400.0
